## Library Dependencies

In [4]:
# python
# coding: utf-8

import os
import subprocess



import webdriver_manager.chrome
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException,WebDriverException
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager

import pandas



## Setup for Automation
    Installing webdriver for automation

In [5]:

path = ChromeDriverManager().install()
path = path.replace("chromedriver.exe", "")
os.environ["PATH"] += os.pathsep + path
driver = webdriver.Chrome()



====== WebDriver manager ======
Current google-chrome version is 98.0.4758
Get LATEST chromedriver version for 98.0.4758 google-chrome
Driver [C:\Users\Shubham\.wdm\drivers\chromedriver\win32\98.0.4758.80\chromedriver.exe] found in cache


## Automation Started
    Get qr scan done from user

In [6]:
driver.get("https://web.whatsapp.com/")

flag = 1
while flag:
    try:
        qrcode = driver.find_element(By.CLASS_NAME, "_2UwZ_")
    except NoSuchElementException:
        flag = 0

Read the excel file from user 

In [7]:
df = pandas.DataFrame()
file = "data.xlsx"
data = pandas.read_excel(file,sheet_name="Sheet1")
numbers = data["Numbers"].to_list()

Utility Functions

In [8]:
# check if that element is accessable or not

def Is_Accessable_Element(xPath):
    flag = 1
    while flag:
        try:
            check = driver.find_element(By.XPATH, xPath)
            flag = 0
        except NoSuchElementException:
            flag = 1

    return xPath


# Convert the available path list into string
def Parse_Paths(Paths):
    ans = ""

    for eachPath in Paths:
        
        ans = ans + str(eachPath)
       
        ans = ans + str(" ")
    str(ans)
    return ans

# Returns the list of all .extension (with absolute path) mention below present in given folder
def GetAll_FilePaths(path):
    fileName = []

    for files in os.listdir(path):
        root, ext = os.path.splitext(files)
        if ext == ".png" or ext==".jpg" or ext == ".mp3" or ext == ".gif" or ext == "jpeg":
            fileName.append(path + str("\\") +files)

    return fileName


Function to send Images and Videos

In [9]:
# Function to send images and videos
def Send_Images_Videos(FilePath):

    # Access the safety pin button
    pin = driver.find_element(By.XPATH,Is_Accessable_Element("/html/body/div[1]/div[1]/div[1]/div[4]/div[1]/footer/div[1]/div/span[2]/div/div[1]/div[2]"))
    pin.click()

    # Access the images and video button
    Image_Icon = driver.find_element(By.XPATH,Is_Accessable_Element("/html/body/div[1]/div[1]/div[1]/div[4]/div[1]/footer/div[1]/div/span[2]/div/div[1]/div[2]/div/span/div[1]/div/ul/li[1]/button"))
    Image_Icon.click()
    

    # Get all the images and videos path from the given directory
    imgList = GetAll_FilePaths(FilePath)
    
    # Convert list of path to a single string (because it is required by subprocess)
    parsePath = Parse_Paths(imgList)
    
    
    #Using Autoit we will shift window focus to the file explorer to send path of images
    time.sleep(1)# THIS SLEEP IS REQUIRED!!!!!!!
    subprocess.run(['AutoIt_Script.exe',parsePath],shell=True)# runs .exe autoit file 
    time.sleep(1)# Don't know whether .exe autoit file will execute parallelly or not


    # Access the send button
    sendButton = driver.find_element(By.XPATH,Is_Accessable_Element("/html/body/div[1]/div[1]/div[1]/div[2]/div[2]/span/div[1]/span/div[1]/div/div[2]/div/div[2]/div[2]/div/div"))
    sendButton.click()
    time.sleep(1) # wait for the button to successfully press

In [10]:
for eachNumber in numbers:
    driver.get(f"https://web.whatsapp.com/send?phone=91{eachNumber}")

    flag = 0
    # wait untill text box appears 
    flag = 1
    number = None



    # wait until contact page is loaded
    while flag:
        try:
            number = driver.find_element(By.XPATH, "/html/body/div[1]/div[1]/div[1]/div[4]/div[1]/header/div["
                                                        "2]/div/div/span")
            flag = 0
        except NoSuchElementException:
            flag = 1



    # check if number is wrong
    try:
        elem = driver.find_element(By.XPATH, "/html/body/div[1]/div[1]/span[2]/div[1]/span/div["
                                                "1]/div/div/div/div/div[2]/div")
        if elem.text == "OK":
            elem.send_keys(Keys.ENTER)
    except NoSuchElementException:
        pass



    # check if correct number is loaded
    try:
        numdata = number.get_property("title")
    except StaleElementReferenceException:
        number = driver.find_element(By.XPATH,
                                        "/html/body/div[1]/div[1]/div[1]/div[4]/div[1]/header/div[2]/div/div/span")
        numdata = number.get_property("title")

    
    # Access the text box for text messages
    text_box = driver.find_element(By.XPATH,Is_Accessable_Element("/html/body/div[1]/div[1]/div[1]/div[4]/div[1]/footer/div[1]/div/span[""2]/div/div[2]/div[1]/div/div[2]"))


    # Sends the all images and videos present in folder
    imgFolder = "D:\PBL_SY_WhatsApp_Automation\whautoman"
    Send_Images_Videos(imgFolder)
    

    

# Before quit wait for 10 secs
# time.sleep(10)  
# driver.quit()